binary and multiclass classification on has_free_cancelation and sustainability level.
clustering on price + distance from center
regression - tree?

In [141]:
import pandas as pd

In [142]:
df = pd.read_csv('../data/hotels_daily_final.csv')
df

,name,location,price,date,stars,distance_from_center,num_external_reviews,booking_user_ratings,avg_user_ratings,airport_taxi,travel_sustainable_level,has_free_cancelation
0,"The Pavilions Amsterdam, The Toren","Amsterdam City Center, Amsterdam","MKD 14,351","['2024-02-01', '2024-02-02']",4.0,0.6 km from center,"1,044 reviews",Excellent,8.9,Limited-time Deal,Travel Sustainable Level 3+,False
1,Sonder Park House,"Oud Zuid, Amsterdam","MKD 8,890","['2024-02-01', '2024-02-02']",4.0,1.7 km from center,"3,190 reviews",Good,7.5,No airport taxi,NaN,True
2,Hotel Jansen Amsterdam Bajeskwartier,"Oost, Amsterdam","MKD 5,267","['2024-02-01', '2024-02-02']",NaN,4.2 km from center,"1,155 reviews",Very Good,8.3,No airport taxi,Travel Sustainable Level 3,True
3,AmicitiA,"Amsterdam City Center, Amsterdam","MKD 1,848","['2024-02-01', '2024-02-02']",NaN,1.2 km from center,"1,820 reviews",Good,7.1,No airport taxi,NaN,True
4,Hotel Washington,"Oud Zuid, Amsterdam","MKD 3,080","['2024-02-01', '2024-02-02']",2.0,2.1 km from center,"2,205 reviews",Good,7.3,No airport taxi,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
54909,Schöne 2 1/2 Zimmer Wohnung im Herzen von Dort...,"Dortmund City Centre, Dortmund","MKD 11,726","['2025-01-31', '2025-02-01']",NaN,0.6 km from center,11 reviews,Excellent,8.6,No airport taxi,NaN,False
54910,schlafgut24,Dortmund,"MKD 7,331","['2025-01-31', '2025-02-01']",NaN,10.9 km from center,16 reviews,Exceptional,9.6,No airport taxi,Travel Sustainable Level 2,False
54911,3 Zimmer Wohnung in Dortmund,Dortmund,"MKD 11,704","['2025-01-31', '2025-02-01']",NaN,3 km from center,3 reviews,Very Good,8.0,No airport taxi,NaN,False
54912,"Wohnung Nähe Uni,Stadion",Dortmund,"MKD 5,211","['2025-01-31', '2025-02-01']",NaN,5 km from center,NaN,NaN,NaN,No airport taxi,NaN,False


# Preprocessing

## Print out the data and see which columns need cleaning

In [143]:
[print(d, df[d].unique()) for d in df.columns]

name ['The Pavilions Amsterdam, The Toren' 'Sonder Park House'
 'Hotel Jansen Amsterdam Bajeskwartier' 'AmicitiA' 'Hotel Washington'
 'Vita Nova' 'Mps Holland' 'Budget Trianon Hotel' 'Belfort Hotel'
 'Amsterdam Hotelboat' 'Hotel Amsterdam Inn' 'The Alfred Hotel'
 'XO Hotel Inner' 'The Concert Hotel' 'De Bedstee Boutique Capsules'
 'Blossoms City Hotel' 'Urban Lodge Hotel' 'Hotel Espresso' 'Felicitas'
 'Hotel Ben Centre' 'MAX Hotel Amsterdam' 'The Flying Pig Uptown'
 'B&B HutSpot' 'Hotel Fogo' 'The Cabin @ MPS Noorderzon'
 'Hotel City Garden Amsterdam'
 'Conscious Hotel Amsterdam City - The Tire Station'
 'ibis Styles Amsterdam City' 'ClinkNOORD Hostel' 'Hotel Parkview'
 'Hotel Allure' 'Hotel Atlantis Amsterdam' 'Hotel New Kit'
 'The Times Hotel' 'Ruby Emma Hotel Amsterdam'
 'The Arcade Hotel Amsterdam' 'Park Inn by Radisson Amsterdam City West'
 'Hampton By Hilton Amsterdam Arena Boulevard'
 'Hampton by Hilton Amsterdam Centre East' 'Boutique Hotel Oosteinde'
 'YOTEL Amsterdam' 'Mercur

[None, None, None, None, None, None, None, None, None, None, None, None]

In [144]:
print("Missing Values:")
print(df.isnull().sum())

Missing Values:
name                            0
location                        0
price                           0
date                            0
stars                       22224
distance_from_center            0
num_external_reviews          528
booking_user_ratings          528
avg_user_ratings             1088
airport_taxi                    0
travel_sustainable_level    13789
has_free_cancelation            0
dtype: int64


Above we can see all the unique values. We can conclude that 

1. the **price** should be converted into Euros and MKD prefix should be removed from the entries. 
2. **stars** column contains Nan values (at least 1), so we should replace them with 0. This would only make sense since nan would represent no stars present, i.e. 0 stars.
3. **distance from center** is represented in string format. We can rename the column and represent the entries in int values, i.e. km from center.
4. **number of external reviews** contains a colon mark and 'reviews' suffix which should both be removed and the column represented as an int column
5. **booking user ratings** also contains dirty data. Nan, "Review score" and user ratings that were incorectly parsed when scraping. These will be converted to numerical values with the help of a dictinary mapping.
6. **airport taxi** contains values that we are not interested in, we will remove these and mark them as Nan.
7. we will also map the **travel sustainable level** column into int values.
8. **avg_user_ratings** is a column of user ratings from (1-10), we can clean this up by applying the average user rating of all the hotels residing in the given area/town.
9. **airport_taxi** is a column defining wether the hotel/appartment has an airport taxi available, we should consider Nan the same as "No airport taxi".
10. **date** column contains a list of 2 dates - arriving and departure dates, we really only need the first date, i.e. the date of stay.

## Clean up the data

#### 1. the **price** should be converted into Euros and MKD prefix should be removed from the entries

In [145]:
mkd_to_euro_coef = 0.01626717
df["price"] = df["price"].str.replace(r'[MKD,]', '', regex=True).str.strip().astype(int) * mkd_to_euro_coef

In [146]:
df['price']

0        233.450157
1        144.615141
2         85.679184
3         30.061730
4         50.102884
            ...    
54909    190.748835
54910    119.254623
54911    190.390958
54912     84.768223
54913    215.865346
Name: price, Length: 54914, dtype: float64

#### 2. **stars** column contains Nan values (at least 1), so we should replace them with 0. This would only make sense since nan would represent no stars present, i.e. 0 stars.

In [147]:
df['stars'].fillna(0,inplace=True)
df['stars'] = df['stars'].astype(int)
df['stars'].unique()

array([4, 0, 2, 3, 1, 5])

#### 3. **distance from center** is represented in string format. We can rename the column and represent the entries in int values, i.e. km from center.

In [148]:
df["distance_from_center"] = df["distance_from_center"].str.extract("([\d.]+)",expand=False).astype(float)
df = df.rename({"distance_from_center": "km_from_center"}, axis="columns")
df["km_from_center"]

0         0.6
1         1.7
2         4.2
3         1.2
4         2.1
         ... 
54909     0.6
54910    10.9
54911     3.0
54912     5.0
54913     1.1
Name: km_from_center, Length: 54914, dtype: float64

#### 4. **number of external reviews** contains a colon mark and 'reviews' suffix which should both be removed and the column represented as an int column

In [149]:
df['num_external_reviews'] = pd.to_numeric(df['num_external_reviews'].str.replace('[^\d]+', '', regex=True), errors='coerce')

In [150]:
df['num_external_reviews'] = df['num_external_reviews'].fillna(0).astype(int)
df["num_external_reviews"]

0        1044
1        3190
2        1155
3        1820
4        2205
         ... 
54909      11
54910      16
54911       3
54912       0
54913       0
Name: num_external_reviews, Length: 54914, dtype: int64

#### 5. **booking user ratings** also contains dirty data. Nan, "Review score" and user ratings that were incorectly parsed when scraping. These will be converted to numerical values with the help of a dictinary mapping.

We will count Review score as a lack of rating and map it the same value as Nan.

In [151]:
import numpy as np
df['booking_user_ratings']= df['booking_user_ratings'].str.extract('([^\d]+)', expand=False).str.strip()
df['booking_user_ratings'] = df['booking_user_ratings'].replace('Review score', np.nan )
df['booking_user_ratings'].unique()

array(['Excellent', 'Good', 'Very Good', nan, 'Wonderful', 'Exceptional'],
      dtype=object)

In [152]:
ratings_mapping = {'Good': 1, 'Very Good': 2, 'Wonderful': 3, 'Exceptional': 4,'Excellent': 5, np.nan: 0}
df['booking_user_ratings'] = df['booking_user_ratings'].map(ratings_mapping)

In [153]:
df['booking_user_ratings'].unique()

array([5, 1, 2, 0, 3, 4])

#### 6. **airport taxi** contains values that we are not interested in, we will replace these and mark them as Nan.

In [154]:
df['airport_taxi'] = df['airport_taxi'].replace(['Limited-time Deal', 'Early 2024 Deal'], np.nan)
df['airport_taxi'].unique()

array([nan, 'No airport taxi', 'Free airport taxi'], dtype=object)

#### 7. we will also map the **travel sustainable level** column into int values.

In [155]:
level_mapping = {
    'Travel Sustainable Level 1': 1,
    'Travel Sustainable Level 2': 2,
    'Travel Sustainable Level 3': 3,
    'Travel Sustainable Level 3+': 4
}
df['travel_sustainable_level'] = df['travel_sustainable_level'].map(level_mapping).fillna(0).astype(int)
df["travel_sustainable_level"].unique()

array([4, 0, 3, 1, 2])

#### 8. handle **avg_user_ratings**. In certain instances, some locations have been designated with the title of their Municipality rather than using the name of the encompassing city to refer to that specific area. We will map these municipalities into the correct cities.

In [156]:
df['location'].unique()

array(['Amsterdam City Center, Amsterdam', 'Oud Zuid, Amsterdam',
       'Oost, Amsterdam', 'Westpoort, Amsterdam', 'Oud West, Amsterdam',
       'Amsterdam Noord, Amsterdam', 'Osdorp, Amsterdam',
       'Geuzenveld-Slotermeer, Amsterdam', 'Zuidoost, Amsterdam',
       'Zeeburg, Amsterdam', 'Sloterdijk, Amsterdam',
       'Antwerp Center, Antwerp', 'Antwerp', 'Berchem, Antwerp',
       'Dortmund City Centre, Dortmund', 'Dortmund',
       'Zuideramstel, Amsterdam', 'Slotervaart, Amsterdam',
       'Bijlmermeer, Amsterdam', 'Borgerhout, Antwerp', 'Amsterdam',
       'Westerpark, Amsterdam', 'Bos en Lommer, Amsterdam',
       'Deurne, Antwerp', 'Wilrijk, Antwerp', 'Hoboken, Antwerp',
       'Merksem, Antwerp', 'De Baarsjes, Amsterdam', 'IJburg, Amsterdam',
       'Turnhout', 'Beerse', 'Merksplas', 'Oud-Turnhout', 'Gierle',
       'Rijkevorsel', 'Vlimmeren', 'Kasterlee', 'Baarle-Hertog'],
      dtype=object)

In [157]:
df['town_location'] = df['location'].str.split().str[-1]
df['town_location'].unique()

array(['Amsterdam', 'Antwerp', 'Dortmund', 'Turnhout', 'Beerse',
       'Merksplas', 'Oud-Turnhout', 'Gierle', 'Rijkevorsel', 'Vlimmeren',
       'Kasterlee', 'Baarle-Hertog'], dtype=object)

In [158]:
municipality_to_city_map = {
    'Turnhout': 'Antwerp',
    'Beerse': 'Antwerp',
    'Merksplas': 'Antwerp',
    'Oud-Turnhout': 'Antwerp',
    'Gierle': 'Antwerp',
    'Rijkevorsel': 'Antwerp',
    'Vlimmeren': 'Antwerp',
    'Kasterlee': 'Antwerp',
    'Baarle-Hertog': 'Antwerp'
}

In [159]:

df['town_location'] = df['town_location'].replace(municipality_to_city_map)

df_mean_rating_by_town = df.groupby(df['town_location'].str.split().str[-1])['avg_user_ratings'].transform('mean')
df['avg_user_ratings'] = df['avg_user_ratings'].fillna(df_mean_rating_by_town)
df['town_location'].unique()

array(['Amsterdam', 'Antwerp', 'Dortmund'], dtype=object)

#### 9. handle **airport_taxi**

In [160]:
df[['airport_taxi']] = df[['airport_taxi']].fillna('No airport taxi') 
df['airport_taxi'] = df['airport_taxi'].map({'Free airport taxi': 1, 'No airport taxi': 0})

#### 10. clean up **date** column so that only 1 date (date of stay) is present

In [161]:
import ast
df['date'] = df['date'].apply(ast.literal_eval)
df['date'] = pd.to_datetime(df['date'].apply(lambda x: x[0]))

### Now we can finaly see our work... and how far we've come

In [162]:
nan_rows = df[df.isnull().any(axis=1)]
nan_rows

,name,location,price,date,stars,km_from_center,num_external_reviews,booking_user_ratings,avg_user_ratings,airport_taxi,travel_sustainable_level,has_free_cancelation,town_location


### The current state of our DF

In [163]:
df

,name,location,price,date,stars,km_from_center,num_external_reviews,booking_user_ratings,avg_user_ratings,airport_taxi,travel_sustainable_level,has_free_cancelation,town_location
0,"The Pavilions Amsterdam, The Toren","Amsterdam City Center, Amsterdam",233.450157,2024-02-01,4,0.6,1044,5,8.900000,0,4,False,Amsterdam
1,Sonder Park House,"Oud Zuid, Amsterdam",144.615141,2024-02-01,4,1.7,3190,1,7.500000,0,0,True,Amsterdam
2,Hotel Jansen Amsterdam Bajeskwartier,"Oost, Amsterdam",85.679184,2024-02-01,0,4.2,1155,2,8.300000,0,3,True,Amsterdam
3,AmicitiA,"Amsterdam City Center, Amsterdam",30.061730,2024-02-01,0,1.2,1820,1,7.100000,0,0,True,Amsterdam
4,Hotel Washington,"Oud Zuid, Amsterdam",50.102884,2024-02-01,2,2.1,2205,1,7.300000,0,0,False,Amsterdam
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54909,Schöne 2 1/2 Zimmer Wohnung im Herzen von Dort...,"Dortmund City Centre, Dortmund",190.748835,2025-01-31,0,0.6,11,5,8.600000,0,0,False,Dortmund
54910,schlafgut24,Dortmund,119.254623,2025-01-31,0,10.9,16,4,9.600000,0,2,False,Dortmund
54911,3 Zimmer Wohnung in Dortmund,Dortmund,190.390958,2025-01-31,0,3.0,3,2,8.000000,0,0,False,Dortmund
54912,"Wohnung Nähe Uni,Stadion",Dortmund,84.768223,2025-01-31,0,5.0,0,0,7.789035,0,0,False,Dortmund


### The last thing we would need to do is divide the df into two seperate df since there are great differences between them.

In [164]:
df_antwerp = df[df['town_location']=='Antwerp']
df_dortmund = df[df['town_location']=='Dortmund']
df_amsterdam = df[df['town_location']=='Amsterdam']

12. We also need to encode 'has_free_cancelation'

In [165]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df['has_free_cancelation'] = label_encoder.fit_transform(df['has_free_cancelation'])

#### Now we create a list of these dfs and apply the methods bellow to the dfs

In [166]:
## now make all code bellow in a function so that i can run in for each df
all_dfs = {"all": df,"Antwerp": df_antwerp, "Dortmund": df_dortmund, "Amsterdam": df_amsterdam}

In [167]:
import papermill as pm

for single_df in all_dfs:
    all_dfs[single_df]["date"]=all_dfs[single_df]["date"].astype(str)
    df_dict = all_dfs[single_df].to_dict(orient='records')
    print(single_df)
    
    pm.execute_notebook(
        './preprocessing_script.ipynb',
        './output_'+single_df+'.ipynb',
        parameters=dict(df=df_dict,df_name=single_df)
    )


Passed unknown parameter: df
Passed unknown parameter: df_name


all


Input notebook does not contain a cell with tag 'parameters'
Executing:   0%|          | 0/37 [00:00<?, ?cell/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Executing: 100%|██████████| 37/37 [03:04<00:00,  4.97s/cell]
/var/folders/tw/dszf86bn0kjgzxp0v_8g35sc0000gn/T/ipykernel_76584/2119138737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs[single_df]["date"]=all_dfs[single_df]["date"].astype(str)
Passed unknown parameter: df
Passed unknown parameter: df_name


Antwerp


Input notebook does not contain a cell with tag 'parameters'
Executing:   0%|          | 0/37 [00:00<?, ?cell/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Executing: 100%|██████████| 37/37 [00:52<00:00,  1.42s/cell]
/var/folders/tw/dszf86bn0kjgzxp0v_8g35sc0000gn/T/ipykernel_76584/2119138737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs[single_df]["date"]=all_dfs[single_df]["date"].astype(str)
Passed unknown parameter: df
Passed unknown parameter: df_name


Dortmund


Input notebook does not contain a cell with tag 'parameters'
Executing:   0%|          | 0/37 [00:00<?, ?cell/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Executing: 100%|██████████| 37/37 [00:50<00:00,  1.36s/cell]
/var/folders/tw/dszf86bn0kjgzxp0v_8g35sc0000gn/T/ipykernel_76584/2119138737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs[single_df]["date"]=all_dfs[single_df]["date"].astype(str)
Passed unknown parameter: df
Passed unknown parameter: df_name


Amsterdam


Input notebook does not contain a cell with tag 'parameters'
Executing:   0%|          | 0/37 [00:00<?, ?cell/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Executing: 100%|██████████| 37/37 [00:52<00:00,  1.41s/cell]
